# Semantic text search using embeddings

We can search through all our reviews semantically in a very efficient manner and at very low cost, by embedding our search query, and then finding the most similar reviews. The dataset is created in the [Get_embeddings_from_dataset Notebook](Get_embeddings_from_dataset.ipynb).

## Installation
Install the Azure Open AI SDK using the below command.

In [1]:
#r "nuget: Azure.AI.OpenAI, *-*"

Installed Packages Azure.AI.OpenAI, 1.0.0-beta.8

In [2]:
#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-tools/nuget/v3/index.json"

Restore sources https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-tools/nuget/v3/index.json

In [3]:
#r "nuget:Microsoft.DotNet.Interactive.AIUtilities, *-*"

using Microsoft.DotNet.Interactive;
using Microsoft.DotNet.Interactive.AIUtilities;

Restore sources https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-tools/nuget/v3/index.json Installed Packages Microsoft.DotNet.Interactive.AIUtilities, 1.0.0-beta.23511.1

In [4]:
var azureOpenAIKey = await Kernel.GetPasswordAsync("Provide your OPEN_AI_KEY");

// Your endpoint should look like the following https://YOUR_OPEN_AI_RESOURCE_NAME.openai.azure.com/
var azureOpenAIEndpoint = await Kernel.GetInputAsync("Provide the OPEN_AI_ENDPOINT");

// Enter the deployment name you chose when you deployed the model.
var deployment = await Kernel.GetInputAsync("Provide deployment name");

### Import namesapaces and create an instance of `OpenAiClient` using the `azureOpenAIEndpoint` and the `azureOpenAIKey`

In [4]:
using Azure;
using Azure.AI.OpenAI;

In [6]:
OpenAIClient client = new (new Uri(azureOpenAIEndpoint), new AzureKeyCredential(azureOpenAIKey.GetClearTextPassword()));

### Loading `Microsoft.Data.Analysis` lastest package

In [5]:
#i "nuget:https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-libraries/nuget/v3/index.json"

Restore sources https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-libraries/nuget/v3/index.json https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-tools/nuget/v3/index.json

In [10]:
#r "nuget: Microsoft.Data.Analysis, *-*"

Restore sources https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-libraries/nuget/v3/index.json https://pkgs.dev.azure.com/dnceng/public/_packaging/dotnet-tools/nuget/v3/index.json Installed Packages Microsoft.Data.Analysis, 0.21.0-preview.23504.1

In [14]:
using Microsoft.ML.Data;
using Microsoft.Data.Analysis;

var df = DataFrame.LoadCsv("fine_food_reviews_with_embeddings_1k.csv");

In [16]:
df.Head(5).Display();

index,Column0,Time,ProductId,UserId,Score,Summary,Text,tokens,embeddings
0,0,1.3511232E+09,B003XPF9BO,A3R7JR3FMEBXQB,5,where does one start...and stop... with a treat like this,Wanted to save some to bring to my Chicago family but my North Carolina family ate all 4 boxes before I could pack. These are excellent...could serve to anyone,34,Dense vector of size 1536
1,1,1.3511232E+09,B003JK537S,A3JBPC3WFUT5ZP,1,Arrived in pieces,"Not pleased at all. When I opened the box, most of the rings were broken in pieces. A total waste of money.",26,Dense vector of size 1536
2,4,1.3511232E+09,B001BORBHO,A1AFOYZ9HSM2CZ,5,Happy with the product,My dog was suffering with itchy skin. He had been eating Natural Choice brand (cheaper) since he was a puppy. I was nervous to change foods. The vet suggested to change foods sand see if the skin issues cleared up. Wellness brand did the job. My dog seems to love the food and the skin issues cleared up within a few weeks.,77,Dense vector of size 1536
3,5,1.3511232E+09,B008PSM0BQ,A3OUFIMGL2K6RS,4,Good Sauce,"This is a good all purpose sauce. Has good flavor that the heat doesn't overpower. Not really that spicy unless you use a whole bunch. 10 good drops is about enough to add a little heat to a pot of soup, but a lot more is needed if you want a lingering burn. Heat isn't quite up to par with other products out there, (such as Spontaneous Combustion) but this has the true aged cayenne hot sauce flavor.",100,Dense vector of size 1536
4,6,1.3511232E+09,B008YA1LQK,A9YEAAQVHFUTX,5,Blackcat,Great coffee! Love all Green Mountain coffee and all the wonderful flavors. Would and do recommend this coffee to all my friends.,27,Dense vector of size 1536


In [19]:
df.Info().Display();

index,Info,Column0,Time,ProductId,UserId,Score,Summary,Text,tokens,embeddings
0,DataType,System.Single,System.Single,System.String,System.String,System.Single,System.String,System.String,System.Single,System.String
1,Length (excluding null values),466,466,466,466,466,466,466,466,466
